In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import json
import torch
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

# chemin vers ton modèle sauvegardé
MODEL_PATH = "./finbert_forex_finetuned"

# 1. Charger le tokenizer et le modèle
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH).to('cuda')

# 2. Charger éventuellement le mapping label (index->texte)
with open(f"{MODEL_PATH}/label_map.json", "r", encoding="utf-8") as f:
    label_map = json.load(f)

print("Label map :", label_map)


Label map : {'0': 'Negative', '1': 'Neutral', '2': 'Positive'}


In [2]:
df = pd.read_csv('./Data/Chatgpt.csv')
df['true_sentiment'] = df['true_sentiment'].apply(lambda x : x.lower())
df.head()

,true_sentiment,paraphrased_title
0,positive,Euro set to gain from the ECB's strong determi...
1,positive,EURCHF's upward trend likely to persist - ING
2,neutral,Is a surge in EURCHF indicating a break above ...
3,positive,EURCHF may prolong its advance towards levels ...
4,positive,EURCHF expected to rise towards 10130 and proj...


In [3]:
def predict(text) : 
    with torch.no_grad() :
        prob = torch.softmax(model(**tokenizer(text, return_tensors="pt").to('cuda')).logits, dim=1).to('cpu')
    sentiment_labels = ['negative', 'neutral', 'positive']
    predicted_sentiment_index = torch.argmax(prob, dim=1).item()
    predicted_sentiment = sentiment_labels[predicted_sentiment_index]
    return prob, predicted_sentiment

def ft_eval(row) :
    text = row['paraphrased_title']
    prob, predicted_sentiment = predict(text)
    return predicted_sentiment

df['predicted_sentiment'] = df.progress_apply(ft_eval, axis=1)
df.head()


100%|██████████| 2264/2264 [00:13<00:00, 164.65it/s]


,true_sentiment,paraphrased_title,predicted_sentiment
0,positive,Euro set to gain from the ECB's strong determi...,positive
1,positive,EURCHF's upward trend likely to persist - ING,positive
2,neutral,Is a surge in EURCHF indicating a break above ...,negative
3,positive,EURCHF may prolong its advance towards levels ...,positive
4,positive,EURCHF expected to rise towards 10130 and proj...,positive


In [4]:
(df['predicted_sentiment'] == df['true_sentiment']).mean()

np.float64(0.7407243816254417)

In [ ]:
def compute_daily_score(list_news) : 
    mean_prob = torch.zeros(3)
    for text in list_news :
        prob, _ = predict(text)
        mean_prob += prob.flatten() / 5
    sentiment_labels = ['negative', 'neutral', 'positive']
    predicted_sentiment_index = torch.argmax(mean_prob).item()
    predicted_sentiment = sentiment_labels[predicted_sentiment_index]
    return mean_prob, predicted_sentiment

list_news = ["GBP/USD rises as US shutdown prolongs, Fed data gap grows",
            "Pound Sterling flattens against US Dollar on absence of US key economic data releases",
            "GBP up marginally and soft – Scotiabank", 
            "GBP/USD may retest the 1.3400 level – UOB Group",
            "GBP/USD strengthens above 1.3430 as traders weigh impact of US government shutdown"]

compute_daily_score(list_news)

(tensor([0.3989, 0.0588, 0.5423]), 'positive')